## Performance Metrics

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_csv("AAPL_clean.csv", parse_dates=['Date'])
df.set_index('Date', inplace=True)

train = df.iloc[:-30]
test = df.iloc[-30:]

def load_forecast(path):
    df = pd.read_csv(path, parse_dates=['Date'])
    df['Forecast'] = pd.to_numeric(df['Forecast'], errors='coerce')
    return df

arima = load_forecast("Models/future_arima.csv")
sarima = load_forecast("Models/future_sarima.csv")
prophet = load_forecast("Models/future_prophet.csv")
lstm = load_forecast("Models/future_lstm.csv")

test_dates = test.index[:30]
test_values = test['Close'].astype(float).values   

models = {
    "ARIMA": arima['Forecast'][:30].astype(float).values,
    "SARIMA": sarima['Forecast'][:30].astype(float).values,
    "Prophet": prophet['Forecast'][:30].astype(float).values,
    "LSTM": lstm['Forecast'][:30].astype(float).values
}

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true = np.array(y_true, dtype=float)  
    y_pred = np.array(y_pred, dtype=float)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

metrics = {}
for model, preds in models.items():
    mae = mean_absolute_error(test_values, preds)
    rmse = np.sqrt(mean_squared_error(test_values, preds))
    mape = mean_absolute_percentage_error(test_values, preds)
    metrics[model] = {"MAE": mae, "RMSE": rmse, "MAPE": mape}

metrics_df = pd.DataFrame(metrics).T
metrics_df = metrics_df.sort_values("RMSE")

print("\n Model Performance (Last 30 Days)")
print(metrics_df)



 Model Performance (Last 30 Days)
               MAE       RMSE      MAPE
ARIMA    10.556540  13.431675  4.494874
SARIMA   10.556540  13.431675  4.494874
LSTM     11.650773  14.631819  4.959445
Prophet  12.700690  14.734440  5.122084
